# UAT Verification for MLIP-AutoPipe Output

This notebook provides a simple, interactive way to verify the output of the MLIP-AutoPipe data generation pipeline. It loads the generated ASE database and performs a series of checks to ensure the data is scientifically valid and consistent with the input configuration.

In [ ]:
from ase.db import connect
import numpy as np
from pathlib import Path

# --- Configuration ---
# IMPORTANT: Change this path to point to the database file generated by your run.
db_path = Path('uat_test_fept.db')
# ----------------------

if not db_path.exists():
    print(f"Error: Database file not found at '{db_path}'. Please run the pipeline first.")
else:
    db = connect(db_path)
    print(f"Successfully connected to the database at '{db_path}'.")
    print(f"Database contains {len(db)} structures.")

## Verification Checks

The following cell loads the first structure from the database and performs several critical checks for physical and chemical validity.

In [ ]:
if 'db' in locals() and len(db) > 0:
    # Retrieve the first structure as an ASE Atoms object
    atoms = db.get_atoms(id=1)
    print("Successfully loaded structure with ID 1.")
    print("Chemical symbols:", atoms.get_chemical_symbols())

    # Check for a reasonable, non-zero cell volume
    assert atoms.get_volume() > 1.0, "Cell volume is zero or negative!"

    # Check for the minimum distance between any two atoms to ensure no overlaps
    # This is a critical check for physical validity
    distances = atoms.get_all_distances(mic=True) # mic=True for periodic boundary conditions
    min_dist = np.min(distances[np.nonzero(distances)])
    print(f"Minimum inter-atomic distance: {min_dist:.2f} Angstrom")
    assert min_dist > 1.0, "Atoms are overlapping, structure is not physically valid!"

    print("\nVerification successful: The structure is valid and has the correct properties.")
else:
    print("Skipping verification checks because the database is not loaded or is empty.")